In [2]:
!pip install -q datasets
#!huggingface-cli login

In [6]:
from datasets import load_dataset
import re

# Load the dataset
dataset = load_dataset('timdettmers/openassistant-guanaco')

# Shuffle the dataset and slice it
dataset = dataset['train'].shuffle(seed=42).select(range(1000))

Repo card metadata block was not found. Setting CardData to empty.


Define a function to transform the data

In [7]:
def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    # Iterate over pairs of segments
    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        # Check if there is a corresponding assistant segment before processing
        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            # Apply the new template
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            # Handle the case where there is no corresponding assistant segment
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}

Apply the transformation

In [8]:
transformed_dataset = dataset.map(transform_conversation)

Pusing it to hub

In [9]:
transformed_dataset.push_to_hub("guanaco-llama2-1k")

HfHubHTTPError: 401 Client Error: Unauthorized for url: https://huggingface.co/api/repos/create (Request ID: Root=1-6682dbf1-138b9a826f585f95020ff53b;ce2b038d-5466-4331-a295-a4551b88baf5)

Invalid username or password.